In [9]:
import random
from abc import ABC, abstractmethod

In [10]:
class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

## Creating LLM

In [11]:
class NakliLLM(Runnable):

    def __init__(self):
        print('LLM created')

    def invoke(self, prompt):
        response_list = ['AI stand for Arti Intelligence.',
                        'Delhi is the capital of India.',
                        'Google is a large US based company.']

        return {'response': random.choice(response_list)}

    def predict(self, prompt):
        response_list = ['AI stand for Arti Intelligence.',
                        'Delhi is the capital of India.',
                        'Google is a large US based company.']
        print('DeprecationWarning: This method is going to be deprecated in next release. Use invoke() instead.')
        return {'response': random.choice(response_list)}

## Creating Prompt Template

In [12]:
class PromptTemplate(Runnable):
    
    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self, input_dict):
        return self.template.format(**input_dict)
    
    def format(self, input_dict):
        print('DeprecationWarning: This method is going to be deprecated in next release. Use invoke() instead.')
        return self.template.format(**input_dict)

In [17]:
llm = NakliLLM()

# llm.predict('How is the weather?')

LLM created


In [18]:
template = PromptTemplate(template='Write a {length} paragraph on {topic}',
                          input_variables=['length','topic'])

## Creating String Parser

In [35]:
class StringParser(Runnable):

    def __init__(self):
        # self.parser = parser
        pass 

    def invoke(self, input_data):
        return input_data['response']

In [36]:
parser = StringParser()

## Creating Runnable Chain

In [37]:
class RunnableConnector(Runnable):

    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            print(f'Initiating Runnable: {runnable}')
            input_data = runnable.invoke(input_data)
        return input_data

In [38]:
chain = RunnableConnector([template, llm, parser])
chain.invoke({'length':'large','topic':'India'})

Initiating Runnable: <__main__.PromptTemplate object at 0x7014741b73e0>
Initiating Runnable: <__main__.NakliLLM object at 0x7014741cad20>
Initiating Runnable: <__main__.StringParser object at 0x7014656a54f0>


'Google is a large US based company.'

## Joke Generation and Explanation 

In [54]:
template1 = PromptTemplate(template='Generate a joke about {topic}',
                           input_variables=['topic'])

template2 = PromptTemplate(template='Explain the joke \n {response}',
                           input_variables=['response'])

model = NakliLLM()

str_parser = StringParser()


chain_1 = RunnableConnector([template1, model])

chain_1.invoke({'topic':'AI'})

LLM created
Initiating Runnable: <__main__.PromptTemplate object at 0x701465655730>
Initiating Runnable: <__main__.NakliLLM object at 0x701465655160>


{'response': 'Google is a large US based company.'}

In [55]:
chain_2 = RunnableConnector([template2, llm, str_parser])

# chain_2.invoke({'joke':'This is a joke'})

#### *We don't have to maunally invoke both the chains*

In [56]:
final_chain = RunnableConnector([chain_1, chain_2])
final_chain.invoke({'topic':'AI'})

Initiating Runnable: <__main__.RunnableConnector object at 0x701465656cf0>
Initiating Runnable: <__main__.PromptTemplate object at 0x701465655730>
Initiating Runnable: <__main__.NakliLLM object at 0x701465655160>
Initiating Runnable: <__main__.RunnableConnector object at 0x701465657a10>
Initiating Runnable: <__main__.PromptTemplate object at 0x701465657530>
Initiating Runnable: <__main__.NakliLLM object at 0x7014741cad20>
Initiating Runnable: <__main__.StringParser object at 0x701465654e30>


'AI stand for Arti Intelligence.'